# 第6章 数据加载、存储与文件格式

## 读取文本格式的数据

- 最常用的处理函数是read_csv

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [2]:
!cat ../../pydata-book/ch06/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
# 默认分隔符为逗号
df = pd.read_csv('../../pydata-book/ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- 当数据没有标题行的时候，可以让pandas使用默认的编号，也可以自己指定

In [4]:
!cat ../../pydata-book/ch06/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
pd.read_csv('../../pydata-book/ch06/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv('../../pydata-book/ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- 还可以将数据中的某一列做成DataFrame的行索引

In [7]:
pd.read_csv('../../pydata-book/ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'], index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


- 如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可

In [8]:
!cat ../../pydata-book/ch06/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [9]:
parsed = pd.read_csv('../../pydata-book/ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

- 如果表格的分隔符不固定，可以使用正则表达式

In [10]:
# 各个字段之间由数量不定的空白符分隔
!cat ../../pydata-book/ch06/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [11]:
# 列名的数量比列的数量少1，因此pandas把第一列自动推断为行索引
pd.read_csv('../../pydata-book/ch06/ex3.txt', sep='\s+')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


- 使用skiprows还可以跳过文件的某些行

In [12]:
!cat ../../pydata-book/ch06/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [13]:
pd.read_csv('../../pydata-book/ch06/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- 文本文件中可以用某些标记来表示缺失值：空字符串、NA、-1.#IND、NULL

In [14]:
!cat ../../pydata-book/ch06/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [15]:
pd.read_csv('../../pydata-book/ch06/ex5.csv')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [16]:
# na_values可以接受一组用于表示缺失值的字符串
pd.read_csv('../../pydata-book/ch06/ex5.csv', na_values=[4, 12])

,something,a,b,c,d,message
0,one,1,2,3.0,NaN,NaN
1,two,5,6,NaN,8.0,world
2,three,9,10,11.0,NaN,foo


In [17]:
# 可以用一个字典为各列指定不同的NA标记值
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('../../pydata-book/ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件

- 在处理很大的文件时，或找出大文件中的参数集以便于后续处理时，你可能只想读取文件的一小部分或逐块对文件进行迭代

In [18]:
result = pd.read_csv('../../pydata-book/ch06/ex6.csv')
result.shape

(10000, 5)

In [19]:
# 可以通过nrows指定读取几行
pd.read_csv('../../pydata-book/ch06/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [20]:
# 设置chunksize，可以逐块对文件进行迭代
chunker = pd.read_csv('../../pydata-book/ch06/ex6.csv', chunksize=1000)
chunker

In [21]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
5    157.0
6    166.0
7    164.0
8    162.0
9    150.0
A    320.0
B    302.0
C    286.0
D    320.0
E    368.0
F    335.0
G    308.0
H    330.0
I    327.0
J    337.0
K    334.0
L    346.0
M    338.0
N    306.0
O    343.0
P    324.0
Q    340.0
R    318.0
S    308.0
T    304.0
U    326.0
V    328.0
W    305.0
X    364.0
Y    314.0
Z    288.0
dtype: float64

## 将数据写出到文本格式

In [22]:
data = pd.read_csv('../../pydata-book/ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [23]:
# 打印到标准输出
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [24]:
# 缺失值在输出结果中默认为空字符串，可以使用na_rep参数指定为别的标记值
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [25]:
# 禁用行标签、列标签
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [26]:
# 只选定某些列
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0
